In [1]:
import pandas as pd
from ivpy import show,attach
from PIL import Image

In [2]:
df = pd.read_csv("/Users/damoncrockett/ivpy-datasets/ycba/metadata.csv")

In [3]:
from keras.applications import ResNet50
from keras.applications import InceptionV3
from keras.applications import Xception # TensorFlow ONLY
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import Model
import numpy as np

/Users/damoncrockett/iv/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Setup model and download weights

In [4]:
MODELS = {
    "vgg16": VGG16,
    "vgg19": VGG19,
    "inception": InceptionV3,
    "xception": Xception, # TensorFlow ONLY
    "resnet": ResNet50
}

In [5]:
modelname = "inception"

In [6]:
inputShape = (224, 224) # for vgg16, which is the modelname default
preprocess = imagenet_utils.preprocess_input

if modelname in ("inception", "xception"):
    inputShape = (299, 299)
    preprocess = preprocess_input

In [7]:
Network = MODELS[modelname]
base_model = Network(weights='imagenet')

96124928/96112376 [==============================] - 3s 0us/step


In [8]:
for layer in base_model.layers:
    print layer.name,layer.output_shape

input_1 (None, None, None, 3)
conv2d_1 (None, None, None, 32)
batch_normalization_1 (None, None, None, 32)
activation_1 (None, None, None, 32)
conv2d_2 (None, None, None, 32)
batch_normalization_2 (None, None, None, 32)
activation_2 (None, None, None, 32)
conv2d_3 (None, None, None, 64)
batch_normalization_3 (None, None, None, 64)
activation_3 (None, None, None, 64)
max_pooling2d_1 (None, None, None, 64)
conv2d_4 (None, None, None, 80)
batch_normalization_4 (None, None, None, 80)
activation_4 (None, None, None, 80)
conv2d_5 (None, None, None, 192)
batch_normalization_5 (None, None, None, 192)
activation_5 (None, None, None, 192)
max_pooling2d_2 (None, None, None, 192)
conv2d_9 (None, None, None, 64)
batch_normalization_9 (None, None, None, 64)
activation_9 (None, None, None, 64)
conv2d_7 (None, None, None, 48)
conv2d_10 (None, None, None, 96)
batch_normalization_7 (None, None, None, 48)
batch_normalization_10 (None, None, None, 96)
activation_7 (None, None, None, 48)
activation_10 (Non

In [9]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

### Run predictions

#### initialize feature matrix

In [10]:
X = pd.DataFrame(columns=range(2048))

In [11]:
for i in df.index:
    if np.random.binomial(1,.0005)==1:
        print i
    
    imagefile = df.imagefile.loc[i]
    
    try:
        image = load_img(imagefile, target_size=inputShape)
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        image = preprocess(image)
        preds = model.predict(image)

        X.loc[i] = preds[0]

        #P = imagenet_utils.decode_predictions(preds)
        #labels = [item[1] for item in P[0]]
        #scores = [str(int(item[2]*100)) for item in P[0]]
    except:
        pass

851
1121
2965
3124
8749
9968
10219
12594
13896
16124
16476
16677
18511
19792
23426
25455
30300


In [12]:
X.to_csv("/Users/damoncrockett/ivpy-datasets/ycba/X.csv",index_label='idx')